# YOUR FIRST LAB
### Please read this section. This is valuable to get you prepared, even if it's a long read -- it's important stuff.

### Also, be sure to read [README.md](../README.md)! More info about the updated videos in the README and [top of the course resources in purple](https://edwarddonner.com/2024/11/13/llm-engineering-resources/)

## Your first Frontier LLM Project

By the end of this course, you will have built an autonomous Agentic AI solution with 7 agents that collaborate to solve a business problem. All in good time! We will start with something smaller...

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, you should have completed the setup linked in the README.

### If you're new to working in "Notebooks" (also known as Labs or Jupyter Lab)

Welcome to the wonderful world of Data Science experimentation! Simply click in each "cell" with code in it, such as the cell immediately below this text, and hit Shift+Return to execute that cell. Be sure to run every cell, starting at the top, in order.

Please look in the [Guides folder](../guides/01_intro.ipynb) for all the guides.

## I am here to help

If you have any problems at all, please do reach out.  
I'm available through the platform, or at ed@edwarddonner.com, or at https://www.linkedin.com/in/eddonner/ if you'd like to connect (and I love connecting!)  
And this is new to me, but I'm also trying out X at [@edwarddonner](https://x.com/edwarddonner) - if you're on X, please show me how it's done 😂  

## More troubleshooting

Please see the [troubleshooting](../setup/troubleshooting.ipynb) notebook in the setup folder to diagnose and fix common problems. At the very end of it is a diagnostics script with some useful debug info.

## If this is old hat!

If you're already comfortable with today's material, please hang in there; you can move swiftly through the first few labs - we will get much more in depth as the weeks progress. Ultimately we will fine-tune our own LLM to compete with OpenAI!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Please read - important note</h2>
            <span style="color:#900;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations. If you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...</span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">This code is a live resource - keep an eye out for my emails</h2>
            <span style="color:#f71;">I push updates to the code regularly. As people ask questions, I add more examples or improved commentary. As a result, you'll notice that the code below isn't identical to the videos. Everything from the videos is here; but I've also added better explanations and new models like DeepSeek. Consider this like an interactive book.<br/><br/>
                I try to send emails regularly with important updates related to the course. You can find this in the 'Announcements' section of Udemy in the left sidebar. You can also choose to receive my emails via your Notification Settings in Udemy. I'm respectful of your inbox and always try to add value with my emails!
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business value of these exercises</h2>
            <span style="color:#181;">A final thought. While I've designed these notebooks to be educational, I've also tried to make them enjoyable. We'll do fun things like have LLMs tell jokes and argue with each other. But fundamentally, my goal is to teach skills you can apply in business. I'll explain business implications as we go, and it's worth keeping this in mind: as you build experience with models and techniques, think of ways you could put this into action at work today. Please do contact me if you'd like to discuss more or if you have ideas to bounce off me.</span>
        </td>
    </tr>
</table>

### If necessary, install Cursor Extensions

1. From the View menu, select Extensions
2. Search for Python
3. Click on "Python" made by "ms-python" and select Install if not already installed
4. Search for Jupyter
5. Click on "Jupyter" made by "ms-toolsai" and select Install if not already installed


### Next Select the Kernel

Click on "Select Kernel" on the Top Right

Choose "Python Environments..."

Then choose the one that looks like `.venv (Python 3.12.x) .venv/bin/python` - it should be marked as "Recommended" and have a big star next to it.

Any problems with this? Head over to the troubleshooting.

### Note: you'll need to set the Kernel with every notebook..

In [1]:
# imports

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
# from openai import OpenAI
from mistralai import Mistral

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI (or Ollama)

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.  

If you'd like to use free Ollama instead, please see the README section "Free Alternative to Paid APIs", and if you're not sure how to do this, there's a full solution in the solutions folder (day1_with_ollama.ipynb).

## Troubleshooting if you have problems:

If you get a "Name Error" - have you run all cells from the top down? Head over to the Python Foundations guide for a bulletproof way to find and fix all Name Errors.

If that doesn't fix it, head over to the [troubleshooting](../setup/troubleshooting.ipynb) notebook for step by step code to identify the root cause and fix it!

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')
api_key = os.getenv('MISTRALAI_API_KEY')

# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
# elif not api_key.startswith("sk-proj-"):
    # print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")



API key found and looks good so far!


# Let's make a quick call to a Frontier model to get started, as a preview!

In [3]:
# To give you a preview -- calling AI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"

messages = [{"role": "user", "content": message}]

messages


[{'role': 'user',
  'content': 'Hello, GPT! This is my first ever message to you! Hi!'}]

In [4]:
"""
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
response.choices[0].message.content
"""

# Mistral AI
model = "ministral-3b-2512"
mistral = Mistral(api_key=api_key)
response = mistral.chat.complete(
    model=model, 
    messages=messages)
response.choices[0].message.content


'Hello, welcome! 😊 It’s so exciting to meet you—your first message to an AI is such a fun milestone! Whether you’re curious about anything from coding to history, science to art, or just want to chat about random fun stuff, I’m here to help.\n\nWhat’s on your mind today? Need a quick tip, a deep dive into a topic, or just someone to brainstorm with? Let’s make this conversation awesome! 🚀\n\n*(P.S. Your first message is already a win—you’re already thinking creatively!)*'

## OK onwards with our first project

In [5]:
# Let's try out this utility

ed = fetch_website_contents("https://edwarddonner.com")
print(ed)

Home - Edward Donner

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Conne

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [6]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [7]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

## Messages

The API from Mistral expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [8]:
messages = [
    {"role": "system", "content": "Tu es un assistant qui répond avec pédagogie qui répond en français"},
    {"role": "user", "content": "Combien font 2 + 2?"}
]

#response = openai.chat.completions.create(model="gpt-4.1-nano", messages=messages)
response = mistral.chat.complete(model=model, messages=messages)
response.choices[0].message.content

"Bien sûr ! Voici comment on peut expliquer cela simplement :\n\n**2 + 2**, c'est comme avoir deux objets identiques (par exemple, deux pommes ou deux balles) et qu'on les ajoute ensemble.\n\n- On commence avec **2** pommes.\n- On en ajoute **2** autres.\n- Maintenant, on a **4** pommes en tout.\n\nDonc, **2 + 2 = 4**.\n\nC’est une opération de base en mathématiques, et c’est un exemple classique pour comprendre les additions. 😊\n\nTu veux que je t’explique autre chose avec un autre exemple ?"

## And now let's build useful messages, using a function

In [9]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [10]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': '\nYou are a snarky assistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\n\nHome - Edward Donner\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n.

## Time to bring it together - the API for Mistral is very simple!

In [11]:
# And now: call the MistralAPI. You will get very familiar with this!

def summarize(url):
    website = fetch_website_contents(url)
    response = mistral.chat.complete(
        model = "ministral-3b-2512",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [12]:
summarize("https://edwarddonner.com")

'**Edward Donner’s AI & Code Haven: A CTO’s Playground**\n\nEdward Donner is basically the guy who built his own AI playground—complete with a Connect Four game for LLMs to duke it out like overgrown toddlers in a board game. He’s the co-founder/CTO of **Nebula.io**, a company that’s basically trying to make recruiters’ lives easier by using AI to find the perfect talent match (and yes, they’ve got patents and awards to prove they’re not just guessing).\n\nHis past life includes founding **untapt**, an AI startup acquired in 2021 (because even startups need to grow up). He’s also got a side hustle of writing blog posts about AI, like:\n- *"The Unique Energy of an AI Live Event"* (probably just him nodding sagely at his own genius).\n- *"AI in Production"* (because AWS needs more people who understand how to scale AI like it’s a well-oiled machine).\n- *"Be an AI Engineer and Leader"* (a curriculum that sounds like it was written by someone who’s never actually built anything).\n\nBonus

In [13]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
display_summary("https://edwarddonner.com")

Oh, *darling*, let’s dive into the delightful chaos of Edward Donner’s digital garden:

Edward Donner is a **code-obsessed, LLM-obsessed, slightly DJ-obsessed** tech whiz who’s basically the human equivalent of a caffeine-fueled, overcaffeinated, slightly confused AI enthusiast. He’s the co-founder/CTO of **Nebula.io**, a company that’s basically trying to make recruiters cry with joy by using AI to find people who are *actually* passionate about their jobs (because, let’s be honest, most people are just there for the paycheck and the free coffee).

His startup, Nebula, specializes in **verticalized LLMs** for talent—because nothing says "human potential" like a well-trained AI that knows how to match people to jobs they’ll actually enjoy (or at least tolerate). They’ve got patents, awards, and a platform that’s basically a love letter to recruiters who finally get that "happy customers" is a real thing.

He’s also got a **Connect Four game** where LLMs go head-to-head in a battle of wits and deception—because nothing says "AI diplomacy" like two bots trying to outsmart each other with lies and half-truths.

And, of course, he’s got a blog full of **tech rants, AI musings, and the occasional DJ ramblings**—because nothing says "amateur electronic music production" like a guy who’s still figuring out how to sync beats and nod sagely at things he doesn’t understand.

**News/Announcements:**
- **November 2025:** Edward’s been hosting AI live events, because nothing says "community building" like a bunch of people pretending to understand AI.
- **September 2025:** Nebula’s diving into **Gen AI and Agentic AI on AWS at scale**, because if there’s one thing AI loves, it’s scaling things it doesn’t fully understand.
- **May 2025:** They’ve got a **curriculum for AI engineers**, because even if you’re not a leader, you can at least pretend you know what you’re doing.
- **May 2025:** Another **AI Executive Briefing**, because executives need to be reminded that AI is the future (even if they’ll never actually use it).

Basically, Edward’s website is a mix of **tech geekery, AI hype, and the occasional existential dread**—all served with a side of slightly amateur DJ energy.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [15]:
display_summary("https://cnn.com")

Oh, *joy*—CNN, the world’s most *engaging* news source, where we all wait patiently for the next crisis to unfold.

**The News:**
- **Ukraine-Russia War & Israel-Hamas War:** Because nothing says "global tension" like two wars happening at once, and we’re all just here to watch the chaos unfold.
- **US Politics:** The usual suspects—Trump, redistricting, and whatever’s next in the 2025 elections (because nothing says "democracy" like a constant election cycle).
- **Business & Tech:** Markets, fear & greed, and whatever’s new in "innovative cities" (probably a new way to charge your phone that doesn’t work).
- **Health & Entertainment:** Because we need to know if our sleep habits are "mindfulness" or just a way to avoid watching another CNN segment.

**The Ad Feedback Form:**
- *"Video player was slow to load content"* – Congrats, CNN, you’ve just invented the world’s most frustrating buffering experience.
- *"Ad was repetitive"* – News? More like *newspeak*—because if you’ve seen one ad, you’ve seen a thousand, and we’re all just here to complain.

**The Verdict:**
CNN is the news equivalent of a bad reality TV show—endless, exhausting, and occasionally entertaining, but mostly just a way to keep you glued to your screen while the world burns.

In [16]:
display_summary("https://anthropic.com")

```markdown
**Anthropic: The AI Company That’s All About the "But Wait, What If?"**

Anthropic is the kind of AI company that treats safety like a strict parent—always pushing boundaries but with a side of "let’s make sure we don’t accidentally invent a dystopian robot overlord." Their flagship product, **Claude**, is a coding and enterprise AI agent that’s basically the Swiss Army knife of AI tools, but with a 4.5-star upgrade (Claude Opus 4.5) that’s so advanced, it might just rewrite the rules of productivity.

They’re not shy about their mission: **"Securing the benefits and mitigating the risks"** of AI for humanity’s long-term well-being. Spoiler: They’re doing this while also trying to convince you that their AI is the most trustworthy thing on the internet. (Spoiler again: It’s not.)

Their latest announcement is **Claude Opus 4.5**, which sounds like it was designed by someone who’s heard of "enterprise workflows" but hasn’t yet mastered the art of not accidentally causing a global blackout. They’re also flexing their developer platform, which is basically the AI equivalent of a backstage pass to a concert you’ll never actually attend.

In short: Anthropic’s AI is sleek, ambitious, and—let’s be honest—probably more interested in making your life easier than in making sure you don’t accidentally start a war. Just don’t blame them if they end up being the reason you finally finish that report.
```

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [22]:
# Step 1: Create your prompts

system_prompt = "Summarize in French the most important to-do tasks of an email in French"
user_prompt = """
Hi,

Please find below a list of high-priority tasks that need your immediate and detailed attention. Given the critical nature of these items and the tight deadlines, I need you to prioritize these above all else today and tomorrow. I expect a preliminary status update by the end of the day today (EOD).

### 1. Q4 Financial Performance Report Compilation

The preliminary Q4 report is due to the executive committee by Monday at 9:00 AM. I need you to own the compilation and final review.

* **Data Aggregation:** Pull the finalized sales figures from the **'FY25_Q4_Final'** sheet in the Shared Drive (Folder: `Finance/Reports/Q4`). Ensure you reconcile the regional data points.
* **Variance Analysis:** Specifically, analyze the variance in the **APAC region's operational expenses** compared to Q3. Provide a clear, concise bulleted explanation for this deviation (max 200 words).
* **Visualization:** Create **three** new charts:
    * Y-o-Y growth comparison for Product A vs. Product B.
    * Monthly gross margin trend for Q4.
    * Headcount efficiency ratio ($Revenue / Employee$) for Q4.
* **Draft Conclusion:** Write a draft of the **"Key Takeaways"** slide (Slide 2 in the template). Focus on the profitability increase despite the slight dip in North American market share.

### 2. Project Phoenix: Milestone 3 Review Preparation

We have the key review meeting with the client next Wednesday. You are responsible for the preparation of all materials.

* **Deck Review:** Go through the latest presentation deck (`Phoenix_M3_Client_V12.pptx`). Check for absolute consistency between the proposed **timeline (Slide 15)** and the **resource allocation matrix (Appendix B)**.
* **Risk Log Update:** Review the **Project Phoenix Risk Register**. Specifically, update the **"Probability"** and **"Impact"** ratings for the "Vendor X supply chain delay" risk. Get final sign-off from Sarah on the new mitigation plan.
* **Action Item Follow-up:** Send a reminder email to the development team (CC me) for the **three pending technical specs** from the last meeting. Require a response within two hours.

### 3. Team Offsite Logistics for January

The offsite planning is behind schedule. Please finalize these specifics immediately.

* **Venue Confirmation:** Call the three short-listed venues (The Summit Center, Lakeside Retreat, and City View Loft). Confirm their availability for the **week of January 20th** and get a formal, itemized quote for **55 people** for two full days, including A/V equipment and lunch.
* **Agenda Draft:** Create a preliminary agenda draft (1 page, bullet points only) for the two days. Allocate a **90-minute slot** for the "Strategic Planning Session" and a **60-minute slot** for the "Team Building Activity."
* **Expense Tracking:** Set up a dedicated, shared expense tracker (Excel or Google Sheets) for the offsite budget. The total budget must not exceed **$15,000 USD**.

**Next Steps & Deadline:**

I need you to send me a concise email update with the following structure by **5:00 PM CET Today**:

1.  Q4 Report: Status (e.g., Data pulled, Variance analysis started)
2.  Project Phoenix: Status (e.g., Deck checked, Sarah contacted)
3.  Offsite Logistics: Status (e.g., Venue calls in progress)

Let me know if you run into any immediate roadblocks.

Best regards,
"""

# Step 2: Make the messages list

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Step 3: Call Mistral
response = mistral.chat.complete(model=model, messages=messages)

# Step 4: print the result
Markdown(response.choices[0].message.content)


Voici un résumé structuré des **tâches prioritaires** à accomplir dans vos emails en français :

---

### **1. Compilation du Rapport de Performance Financaire Q4**
**Échéance** : Soumettre le rapport au comité exécutif avant **9h00 lundi**.

#### **Actions immédiates** :
- **Réunion des données** :
  - Extraire les chiffres de vente définitifs depuis la feuille **`FY25_Q4_Final`** (dans le dossier : `Finance/Reports/Q4`).
  - Vérifier la cohérence des données régionales.

- **Analyse des écarts** :
  - Étudier la variance des **dépenses opérationnelles APAC** par rapport au Q3.
  - Rédiger une explication claire et concise (max 200 mots) des causes de ce décalage.

- **Création de visualisations** :
  - **3 graphiques** :
    1. Comparaison année sur année (YoY) du **croissance de Product A vs. Product B**.
    2. Tendance mensuelle des **marges brutes Q4**.
    3. Ratio d’efficacité de la main-d’œuvre (**$Revenu / Employé**) pour Q4.

- **Conclusion préliminaire** :
  - Rédiger un **texte pour la slide "Points Clés"** (Slide 2 du template).
  - Insister sur la **hausse de la rentabilité**, malgré la baisse de la part de marché en Amérique du Nord.

---

### **2. Préparation de la Revue du Milestone 3 – Projet Phoenix**
**Échéance** : Réunion client le **mercredi prochain** → Préparer tous les supports.

#### **Actions prioritaires** :
- **Revue du deck** :
  - Vérifier la **cohérence** entre :
    - Le **calendrier proposé** (Slide 15).
    - La **matrice d’allocation des ressources** (Annexe B).

- **Mise à jour du registre des risques** :
  - Mettre à jour les **notes de probabilité et d’impact** pour le risque **"Retard de la chaîne d’approvisionnement de Vendor X"**.
  - Obtenir le **signe bon** de Sarah sur le nouveau plan de mitigation.

- **Suivi des actions** :
  - Envoyer un **relance** à l’équipe développement (CC moi) pour les **3 spécifications techniques manquantes** de la dernière réunion.
  - **Exiger une réponse sous 2h**.

---

### **3. Logistique du Séminaire d’Équipe – Janvier**
**Problème** : Retard dans la planification → Finaliser **dès maintenant**.

#### **Actions urgentes** :
- **Confirmer le lieu** :
  - Appeler les **3 sites retenus** :
    - The Summit Center, Lakeside Retreat, City View Loft.
  - Vérifier leur disponibilité pour la **semaine du 20 janvier** et obtenir un **devis détaillé** pour **55 personnes** (2 jours complets, équipement A/V + repas).

- **Créer un agenda préliminaire** :
  - Rédiger un **plan en 1 page** (points bulles uniquement) pour les 2 jours.
  - Prévoir :
    - **90 min** pour la **"Session de Planification Stratégique"**.
    - **60 min** pour l’**"Activité de Cohésion d’Équipe"**.

- **Suivi budgétaire** :
  - Créer un **tableau partagé** (Excel/Google Sheets) pour suivre les dépenses.
  - **Budget maximal : 15 000 USD**.

---

### **Prochaines Étapes & Retour**
**À envoyer avant 17h00 CET aujourd’hui** :
1. **Q4 Report** : État d’avancement (ex. : "Données récupérées / Analyse des écarts en cours").
2. **Projet Phoenix** : État (ex. : "Deck vérifié / Sarah contactée").
3. **Logistique Offsite** : État (ex. : "Appels en cours / Budget validé").

---
**À signaler immédiatement** : Tout blocage ou difficulté.

---
*Format concis et priorisé pour une action efficace.*

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293